In [1]:
from mlxtend.regressor import StackingRegressor

import pandas_profiling
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
import statsmodels.api as sm # import statsmodels 
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
#https://scikit-learn.org/stable/modules/model_evaluation.html
#todo: add adjusted r2
from sklearn.metrics import explained_variance_score,mean_absolute_error,mean_squared_error,mean_squared_log_error,r2_score
import catboost as cb
from sklearn.preprocessing import StandardScaler
import os
from xgboost import XGBRegressor
import pickle

Using TensorFlow backend.


XGBoostError: XGBoost Library (xgboost.dll) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libgomp.so for UNIX-like OSes)
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['[WinError 127] The specified procedure could not be found']


In [ ]:
def evalRegressor(model, x_test, y_test):
    y_pred = model.predict(x_test)
    evs = explained_variance_score(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    mse = mean_squared_error(y_test,y_pred)
    msl = 0
    r2 = r2_score(y_test,y_pred)
    Ar2 = 1-(1-r2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
    return y_pred,evs,mae,mse,msl,r2,Ar2

64


In [ ]:
df = pd.read_csv('cleanData.csv')

In [ ]:
del df['price']
del df['availability_30']
del df['availability_365']
del df['availability_60']
del df['availability_90']
del df['Occupy']

In [ ]:
dfNoNA = df.fillna(df.mean())

In [ ]:
XNoNA = dfNoNA.drop(['Income_per_month'], axis=1)
yNoNA = dfNoNA['Income_per_month']

In [ ]:
X_trainNoNA, X_testNoNA, y_trainNoNA, y_testNoNA = train_test_split(XNoNA, yNoNA)


In [ ]:
X = df.drop(['Income_per_month'], axis=1)
y = df['Income_per_month']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [ ]:

lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')

stregr = StackingRegressor(regressors=[svr_lin, lr, ridge], 
                           meta_regressor=svr_rbf)

# Training the stacking classifier

stregr.fit(X, y)
stregr.predict(X)

# Evaluate and visualize the fit

print("Mean Squared Error: %.4f"
      % np.mean((stregr.predict(X) - y) ** 2))
print('Variance Score: %.4f' % stregr.score(X, y))

with plt.style.context(('seaborn-whitegrid')):
    plt.scatter(X, y, c='lightgray')
    plt.plot(X, stregr.predict(X), c='darkgreen', lw=2)

plt.show()

In [ ]:
# Initializing models
RF = RandomForestRegressor()
XGB = XGBRegressor(nthread=-1)
CB = cb.CatBoostRegressor()
svr_lin = SVR(kernel='linear')
svr_rbf = SVR(kernel='rbf')
regressors = [RF,svr_lin,XGB,CB,]
stregr = StackingRegressor(regressors=regressors, 
                           meta_regressor=svr_rbf)


params = {  'RF__max_depth': [3, 4, 5, None,10],
            'RF__n_estimators': [10, 100, 200, 400,600,1000],
            'RF__max_features': [2,4,6,10,15,30,50],
            'RF__min_samples_leaf': [1,3,5,10,50,30],
            'CB__depth':[3,1,6,10],
            'CB__iterations':[100],
            'CB__learning_rate':[0.0001,0.03,0.001,0.01,0.1,0.3], 
            'CB__l2_leaf_reg':[3,1,5,10,40],
            'CB__border_count':[32,5,20,50,100],
          #'ctr_border_count':[50,5,10,20,100,200],
            'CB__thread_count':[4],
            'XGB__min_child_weight':[3,4,5], 
            'XGB__gamma':[i/10.0 for i in range(1,6)],
            'XGB__subsample':[i/10.0 for i in range(6,11)],
            'XGB__colsample_bytree':[i/10.0 for i in range(6,11)],
            'XGB__max_depth': [4,5,6]
            'meta_regressor__C': [0.1, 1.0, 10.0, 100.0],
            'meta_regressor__gamma': [0.1, 1.0, 10.0]}

grid = GridSearchCV(estimator=stregr, 
                    param_grid=params, 
                    cv=5,
                    refit=True)
grid.fit(X, y)

In [ ]:
modelXGB = pickle.load(open('Final_CLF_XGB.sav','rb'))
modelRF = pickle.load(open('Final_CLF_RF.sav','rb'))
modelCAT = pickle.load(open('Final_CLF_CAT.sav','rb'))


In [ ]:
predXGB = modelXGB.predict(x_trainNoNA)
predRF = modelRF.predict(x_trainNoNA)
predCAT = modelCAT.predict(x_trainNoNA)

In [ ]:
preds = pd.DataFrame(
    {'predXGB': predXGB,
     'predRF': predRF,
     'predCAT': predCAT
    })

In [ ]:
svr_rbf = SVR(kernel='rbf')
svr_rbf.fit(preds, y_trainNoNA)

In [ ]:
preds_test = pd.DataFrame(
    {'predXGB': modelXGB.predict(x_testNoNA),
     'predRF': modelRF.predict(x_testNoNA),
     'predCAT': modelCAT.predict(x_testNoNA)
    })
y_pred_XGB,evs_XGB,mae_XGB,mse_XGB,msl_XGB,r2_XGB,Ar2_XGB = evalRegressor(svr_rbf,preds_test, y_testNoNA)


In [ ]:
y_pred_XGB,evs_XGB,mae_XGB,mse_XGB,msl_XGB,r2_XGB,Ar2_XGB